### Rental Simulation
---

Simulate income of real-estate investment using
- Mortgage rate
- Price-to-rent ratio
- State tax
- Selling transaction fee
- Simulated housing price growth

In [219]:
import pandas as pd
from typing import Tuple, List, Sequence, Union

class HousingMarket(object):
    
    def __init__(self, mkt_value=None, monthly_rent=None, growth=0.05, 
                 price_to_rent=None, rental_tax_n_depreciation=0.1, capital_gain=0.2, 
                 transaction_fee=0.05, mortgage_rate=0.04, property_tax=0.015,
                 interest_rate=0.015, mortgage_fee=0.01, downpayment=0.03,
                 management_fee=0.08):
        """
        We prioritize calculating results given mkt_value and monly_rent
        if not provided, default housing value is 500,000
        
        For price_to_rent, values can be None, float, or (float, float)
        - None: look at mkt_value and monly_rent
        - float: assume price_to_rent stays constant
        - (float, float): (start ptr, end prt), and assume linear growth
        """
        self.mkt_value = mkt_value
        self.monthly_rent = monthly_rent  # initial monthly_rent
        self.growth = growth
        if type(price_to_rent) == tuple:
            self.price_to_rent = price_to_rent 
        elif str(price_to_rent).isnumeric():
            self.price_to_rent = (price_to_rent, price_to_rent)
        else:
            self.price_to_rent = None
        self.capital_gain = capital_gain
        self.transaction_fee = transaction_fee
        self.mortgage_rate = mortgage_rate
        self.property_tax_annual = property_tax
        self.mortgage_fee = mortgage_fee
        self.downpayment = downpayment
        self.management_fee = management_fee 
        self.rental_tax = rental_tax_n_depreciation
        self.interest_rate = interest_rate
        
        if mkt_value is None and monthly_rent is None and price_to_rent is None:
            print("No values provided. Either provide (mkt_value, monthly_rent) or price_to_rent.")
            return
        
        if not (mkt_value is None and monthly_rent is None):
            ptr = mkt_value / (12.0 * monthly_rent)
            self.price_to_rent = (ptr, ptr)
        elif price_to_rent is not None:
            self.mkt_value = 5e5  # set default market value
            
        self.captial = self.mkt_value * downpayment

        self.print_args()
    
    def run_simulation(self, years=30):
        self.years = years
        # initialize df
        df = pd.DataFrame({"month": range(1, 13)})
        df = pd.concat([df.copy() for _ in range(years)])
        df.reset_index(drop=True, inplace=True)
        df["years"] = df.index // 12 + 1
        
        # insert values
        df["growth"] = (self.growth+1)**(1/12)-1
        df["cum_growth"] = (df.growth + 1).cumprod()
        df["market_value"] = self.mkt_value * df["cum_growth"]
        df["ptm_ratio"] = self.calc_running_ptm_ratio(df, self.price_to_rent)
        df["monthly_rent"] = df["market_value"] / df["ptm_ratio"] / 12 
        self.pretax_df = df.copy()  # checkpointing
        
        # mortgage payments
        df = self.mortgage_payments(df)
        
        # --- spendings --- 
        df["monthly_spending"] = df["monthly_rent"] * self.management_fee
        df["monthly_spending"] = (df["monthly_rent"] - df["monthly_mortgage"]) * self.rental_tax
        df["monthly_spending"] += df[["month", "market_value"]].apply(self.calc_property_tax, axis=1) 
        # selling agent fee
        df.loc[df.index[-1], "monthly_spending"] += df.loc[df.index[-1], "market_value"] * self.transaction_fee 
        df.loc[df.index[-1], "monthly_spending"] += df.loc[df.index[-1], "market_value"] * self.capital_gain
        
        # net income
        df["net_income"] = df["monthly_rent"] - df["monthly_mortgage"] - df["monthly_spending"]
        results = self.calc_return(df)
        return df
    
    def calc_return(self, df):
        results = {}
        r = (1 + self.interest_rate / 12)
        df["discount_factor"] = r ** (df.index + 1)
        df["discounted_net_income"] = df["net_income"] / df["discount_factor"]
        total_ret = df["discounted_net_income"].sum()
        cap_gain = df.loc[len(df), "mkt_value"]*(1-self.transaction_fee)*(1-self.capital_gain)
        cap_gain /= df.loc[len(df), "discount_factor"]

#         annual_rental_return = df[["market_value", "net_income"]].apply()
        # calculate annual rental return. It's roughly 4% accourding to Randy
        return 
    
    def mortgage_payments(self, df):
        p = self.mkt_value * (1-self.downpayment) * (1+self.mortgage_fee)
        rate_m = self.mortgage_rate / 12
        n = len(df)
        df["monthly_mortgage"] = p*rate_m*(1+rate_m)**n/((1+rate_m)**n-1)
        return df
    
    def calc_property_tax(self, row):
        if row["month"] == 12:
            return row["market_value"] * self.property_tax_annual
        return 0
    
    def calc_running_ptm_ratio(self, df, ptr: Tuple):
        # ptr - price to rent ratio 
        assert(len(ptr) == 2)
        start, end = ptr
        df["ptr_ratio_temp_"] = start
        df["ptr_ratio_temp_"] += (end-start)/(len(df)-1)*df.index
        result = df["ptr_ratio_temp_"].copy()
        df.drop("ptr_ratio_temp_", axis=1, inplace=True)
        return result
    
    def print_args(self):
        print("Args provided:")
        df = pd.DataFrame(vars(self)).T
        df.columns = ["values"] + [str(i) for i in range(len(df.columns)-1)]
        print(df["values"].to_frame())
        

In [220]:
HM = HousingMarket(price_to_rent=(10,20))
df = HM.run_simulation()

Args provided:
                     values
capital_gain            0.2
captial               15000
downpayment            0.03
growth                 0.05
interest_rate         0.015
management_fee         0.08
mkt_value            500000
monthly_rent           None
mortgage_fee           0.01
mortgage_rate          0.04
price_to_rent            10
property_tax_annual   0.015
rental_tax              0.1
transaction_fee        0.05


In [221]:
df.net_income

0        1660.521022
1        1665.359513
2        1670.233350
3        1675.142492
4        1680.086901
5        1685.066538
6        1690.081368
7        1695.131356
8        1700.216469
9        1705.336675
10       1710.491944
11      -6159.317753
12       1720.907555
13       1726.167843
14       1731.463085
15       1736.793259
16       1742.158340
17       1747.558308
18       1752.993144
19       1758.462828
20       1763.967343
21       1769.506673
22       1775.080803
23      -6488.060280
24       1786.333410
25       1792.011863
26       1797.725068
27       1803.473016
28       1809.255699
29       1815.073112
           ...      
330      5400.714792
331      5420.371201
332      5440.094937
333      5459.886212
334      5479.745242
335    -23901.296298
336      5519.667424
337      5539.731009
338      5559.863214
339      5580.064257
340      5600.334357
341      5620.673734
342      5641.082610
343      5661.561206
344      5682.109745
345      5702.728450
346      5723

In [222]:
df

,month,years,growth,cum_growth,market_value,ptm_ratio,monthly_rent,monthly_mortgage,monthly_spending,net_income,discount_factor
0,1,1,0.004074,1.004074,5.020371e+05,10.000000,4183.642182,2338.618825,184.502336,1660.521022,1.001250
1,2,1,0.004074,1.008165,5.040824e+05,10.027855,4189.018284,2338.618825,185.039946,1665.359513,1.002502
2,3,1,0.004074,1.012272,5.061361e+05,10.055710,4194.433658,2338.618825,185.581483,1670.233350,1.003755
3,4,1,0.004074,1.016396,5.081982e+05,10.083565,4199.888261,2338.618825,186.126944,1675.142492,1.005009
4,5,1,0.004074,1.020537,5.102686e+05,10.111421,4205.382048,2338.618825,186.676322,1680.086901,1.006266
5,6,1,0.004074,1.024695,5.123475e+05,10.139276,4210.914978,2338.618825,187.229615,1685.066538,1.007523
6,7,1,0.004074,1.028870,5.144349e+05,10.167131,4216.487012,2338.618825,187.786819,1690.081368,1.008783
7,8,1,0.004074,1.033062,5.165308e+05,10.194986,4222.098110,2338.618825,188.347928,1695.131356,1.010044
8,9,1,0.004074,1.037270,5.186352e+05,10.222841,4227.748235,2338.618825,188.912941,1700.216469,1.011306
9,10,1,0.004074,1.041496,5.207482e+05,10.250696,4233.437353,2338.618825,189.481853,1705.336675,1.012571
